In [ ]:
from math import factorial

# число каналов
k = 5

# 600 байт — м.о. длины пакета
# 2.3 сообщ/сек — интенсивность входящего потока
lmd = 5*600

# 1024 байт/сек — интеснсивность обслуживания
mu = 1024 * k

#первое распредление Эрланга
def Erlang_distribution(lmd, mu, k):
    ro = lmd/mu
    p = []
    for n in range(k+1):
        p.append((ro**n / factorial(n)) /(sum([ro**m / factorial(m) for m in range(k+1)])))
    return p

p=Erlang_distribution(lmd, mu, k)
print("Стационарное распределение:")
[print(f'p{p.index(item)} = {item:.3f}') for item in p]

h = sum([i * p[i] for i in range(k+1)]) + k * sum([p[i] for i in range(k+1, len(p))])
print(f'Математическое ожидание числа занятых приборов: {h:.3f}')
print(f'Вероятность потери пакета: {p[4]:.3f}')

In [1]:
import numpy as np

def get_omega(_L, _Theta):
    _omega = np.array([1/_L for _ in range(_L)])
    for _ in range(1000000):
        _omega = _omega.dot(_Theta)
    return _omega

def get_A(_kappa, _L, _N):
    _A = np.ones((_L, _N + 1))
    for i in range(_L):
        for m in range(1, _N):
            tmp = 1
            for l in range(1, m + 1):
                tmp *= min(l, _kappa[i]) 
            _A[i][m] = tmp
    return _A

def get_G(_L, _N, _x, _A):
    _G = np.zeros((_N+1, _L))
    for i in range(_N + 1):
        _G[i][0] = _x[0]**i
    _G[0] = [1]*_L    
    for i in range(1, _N + 1):
        for j in range(1, _L):
            # изменения для системы вида M/M/N
            if j == 1:
                _G[i][j] = _G[i][j-1] + _x[j]**i / _A[j][i] * _G[i-1][j]
            else:
                _G[i][j] = _G[i][j-1] + _x[j] * _G[i-1][j]
    return _G

P = 0.5
F = [0.08, 0.05]
L = 9
mu = [10, 100, 10000, 100000, 120000, 1000, 1500, 10000, 12000]
N = 10
Theta = np.array([
 [.0, 1., .0, .0, .0, .0, .0, .0, .0],
 [.0, .0, 1., .0, .0, .0, .0, .0, .0],
 [.0, .0, .0, P, 1-P, .0, .0, .0, .0],
 [.0, .0, .0, .0, .0, F[0], .0, 1-F[0], .0],
 [.0, .0, .0, .0, .0, .0, F[1], .0, 1-F[1]],
 [.0, .0, .0, 1., .0, .0, .0, .0, .0],
 [.0, .0, .0, .0, 1., .0, .0, .0, .0],
 [1., .0, .0, .0, .0, .0, .0, .0, .0],
 [1., .0, .0, .0, .0, .0, .0, .0, .0]
])

kappa = np.ones(L)
kappa[1] = N

omega = get_omega(L, Theta)
x = omega / mu
A = get_A(kappa, L, N)
G = get_G(L, N, x, A)
n = np.zeros(L)
h = np.zeros(L)
lambd = np.zeros(L)
u = np.zeros(L)
    
for i in range(L):
    n[i] = sum([x[i] ** m * G[N - m][L-1] / G[N][L-1] for m in range(1, N+1)])
    h[i] = x[i] * G[N-1][L-1] / G[N][L-1]
    lambd[i] = h[i] * mu[i]
    u[i] = n[i] / lambd[i]
    print(f"М.о. длительности пребывания требований в {i} системе обслуживания {u[i]}")

М.о. длительности пребывания требований в 0 системе обслуживания 1.0097480330662203
М.о. длительности пребывания требований в 1 системе обслуживания 0.01111111112032348
М.о. длительности пребывания требований в 2 системе обслуживания 0.00010010010010010009
М.о. длительности пребывания требований в 3 системе обслуживания 1.0000543507799335e-05
М.о. длительности пребывания требований в 4 системе обслуживания 8.333698846440633e-06
М.о. длительности пребывания требований в 5 системе обслуживания 0.0010004349717268378
М.о. длительности пребывания требований в 6 системе обслуживания 0.0006667836462537287
М.о. длительности пребывания требований в 7 системе обслуживания 0.00010005002501250627
М.о. длительности пребывания требований в 8 системе обслуживания 8.336807002917883e-05


In [1]:
from pprint import pprint
import numpy as np
from decimal import *

def stationary_distribution(omega, theta, eps):
    while np.linalg.norm(omega.dot(theta) - omega) > eps:
        omega = omega.dot(theta)
    return omega

def get_A(kappa, L, N):
    A = np.ones((L, N + 1))
    for i in range(L):
        for m in range(1, N):
            tmp = 1
            for l in range(1, m + 1):
                tmp *= min(l, kappa[i])
            A[i][m] = tmp
    return A

# нормализующие константы
def get_G(L, N, x):
    G = np.zeros((N+1, L))
    for i in range(N + 1):
        G[i][0] = x[0]**i
    G[0] = [1]*L
    for i in range(1, N + 1):
        for j in range(1, L):
            # изменения для системы вида M/M/N
            if j == 1:
                G[i][j] = G[i][j-1] + x[j]**i / A[j][i] * G[i-1][j]
            else:
                G[i][j] = G[i][j-1] + x[j] * G[i-1][j]
    return G

L = 9 # 8 M/M/1 + 1 M/M/N
N = 13 # число требований в сети
kappa = np.ones(L)
kappa[1] = N

mu = np.array([10, 100, 10000, 100000, 1000, 10000, 120000, 1500, 12000])

theta = np.array([
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, .5, 0, 0, .5, 0, 0],
    [0, 0, 0, 0, .08, .92, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, .05, .95],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0]
])

# нахождение вектора omega
_omega = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0])
eps = 0.0001
omega = stationary_distribution(_omega, theta, eps)
print(f'Omegas: {omega},\nCheck (~1): {sum(omega)}')

x = omega / mu
print(f'x = {x}')
A = get_A(kappa, L, N)
print(f'A = ')
pprint(A)
G = get_G(L, N, x)
print(f'G = ')
pprint(G)
# вероятности что в системах m и более требований
p = np.zeros((N+1, L))
# вероятности что в системах ровно m требований
p_m = np.zeros((N+1,L))
# м.о. числа требований в системах
s = np.zeros(L)
# м.о. числа занятых приборов в системах
h = np.zeros(L, dtype=Decimal)
# интенсивности входного потока требований в системах
lmbds = np.zeros(L)
# м.о. длительности пребывания требований в системах
u = np.zeros(L)

for i in range(N + 1):
    for j in range(L):
        p[i][j] = x[j]**i * (G[N-i][L-1] / G[N][L-1])
for i in range(N + 1):
    for j in range(L):
        p_m[i][j] = (x[j]**i / G[N][L-1]) * (G[N-i][L-1] - x[j] * G[N-i-1][L-1])
for i in range(L):
    for j in range(1, N + 1):
        s[i] += x[i]**j * (G[N-j][L-1] / G[N][L-1])
    getcontext().prec = 50
    h[i] = Decimal(x[i]) * (Decimal(G[N-1][L-1]) / Decimal(G[N][L-1]))
    lmbds[i] = h[i] * mu[i]
    u[i] = s[i] / lmbds[i]
print(f'lambdas = {lmbds}')
# математическое ожидание длительности пребывания требований
# в системах обслуживания
print('\n\nМ.о. длительности пребывания требований в системах:\n')
for i in range(L):
    print(f'Система {i + 1}: {u[i]}')

Omegas: [0.19458823 0.19457215 0.19454098 0.1057647  0.00845793 0.09726616
 0.10242514 0.00511924 0.09726547],
Check (~1): 1.0000000000000009
x = [1.94588230e-02 1.94572155e-03 1.94540980e-05 1.05764699e-06
 8.45792709e-06 9.72661616e-06 8.53542812e-07 3.41282367e-06
 8.10545622e-06]
A = 
array([[1.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        1.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        1.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        1.000000e+00, 1.000000e+00],
       [1.000000e+00, 1.000000e+00, 2.000000e+00, 6.000000e+00,
        2.400000e+01, 1.200000e+02, 7.200000e+02, 5.040000e+03,
        4.032000e+04, 3.628800e+05, 3.628800e+06, 3.991680e+07,
        4.790016e+08, 1.000000e+00],
       [1.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        1.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        1.000000e+00, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        1.000000e+00, 1.000000e+00],
       [1.000000e+00, 1

In [ ]:
import numpy as np
from math import sqrt

def get_e(_N, _theta):
    e_array = np.array([1/_N for _ in range(_N)])
    for _ in range(1000000):
        e_array = e_array.dot(_theta)
    return e_array

def get_K_over(_K, _N, _lambd, _e, _A):
    summ = 0
    for i in range(_N):
        summ += _lambd * _e[i] * _A
    return _K - summ

def get_D(_lambd, _e, _A, _k, _N):
    D_array = np.zeros(_N)
    for i in range(_N):
        D_array[i] = _lambd * _e[i] * (_A * _lambd * _e[i] * _k + 1)
    return D_array
    

def get_optimized_C(_C, _mu_array, _N):
    summ = 0
    for i in range(_N):
        summ += _C[i] * _mu_array[i]
    return summ

def get_optimized_mu(_D, _B, _C, _K_over, _N, _k, _lambd, _e):
    mu_array = np.zeros(_N)
    for i in range(_N):
        tmp_1 = sqrt(_D[i] / (_B * _C[i]))
        summ = 0
        for j in range(_N):
            summ += sqrt((_C[j] * _D[j]) / _B)
        tmp_2 = summ / _K_over
        tmp_3 = _k * (lambd * _e[i]) / _B
        mu_array[i] = tmp_1 * tmp_2 + tmp_3
    return mu_array
    
    
Theta = np.array([
    [0, 0.2, 0, 0.5, 0.3],
    [1, 0, 0, 0, 0],
    [1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [1, 0, 0, 0, 0]
])

N = 5 #число систем
K = 10 #число требований
k = (K - 1)/K
omega = 8
psi = 10
lambd = 0.7 #интенсивность поступления 
C = np.array([11, 18, 14, 10, 5]) 
mu = np.ones(5)

# Решение уравнений e*p = e с условием нормировки
e = get_e(N, Theta)
print('e_array:', e)
print('Условие нормировки:', sum(e))

# Вычисление характеристик систем сети

A = omega / psi ** 2
B = (1 / omega) / ((1 / omega) + (1 / psi))
K_over = get_K_over(K, N, lambd, e, A)
D = get_D(lambd, e, A, k, N)
    
mu_array = get_optimized_mu(D, B, C, K_over, N, k, lambd, e)
print(f'mu_array = {mu_array}')
optimized_C = get_optimized_C(C, mu_array, N)
print(f'optimized_C = {optimized_C}')

In [1]:
import numpy as np

def get_new_mu(_s, _sigma, _mu, _s_0, _h):
    new_mu = _s
    for i in range(len(_s)):
        for j in range(3):
            if(_s[i][j] + _sigma[j] > _s_0[j]) and (_mu[j] <= 1):
                new_mu[i][j] = (_s[i][j] + _sigma[j] - _s_0[j]) / _h[j]
            elif(_s[i][j] + _sigma[j] > _s_0[j]) and (_mu[j] > 1):
                new_mu[i][j] = 1
            else:
                new_mu[i][j] = 0
    return new_mu

mu = np.array([0.3, 0.4, 0.5])

theta = np.array([
    [0, 0.3, 0.7],
    [0.2, 0, 0.8],
    [0.4, 0.6, 0]
])

s = np.array([
    [0, 1, 2], [0, 2, 1],
    [1, 2, 0], [1, 0, 2], [1, 1, 1],
    [2, 0, 1], [2, 1, 0],
    [3, 0, 0], [0, 3, 0], [0, 0, 3]
])

s_0 = np.array([1, 1, 1,])
nu = 3.
h = nu / mu
nu_arr = theta * nu

sigma = np.zeros(3)
for i in range(len(theta)):
    for j in range(len(theta)):
        sigma[i] += nu_arr[i][j]
mu_new = get_new_mu(s.astype(float), sigma, mu, s_0.astype(float), h)
for i in range(len(s)):
    print(f"Для состояния {s[i]} мин. интенсивности обслуживания = {mu_new[i]}")

Для состояния [0 1 2] мин. интенсивности обслуживания = [0.2        0.4        0.66666667]
Для состояния [0 2 1] мин. интенсивности обслуживания = [0.2        0.53333333 0.5       ]
Для состояния [1 2 0] мин. интенсивности обслуживания = [0.3        0.53333333 0.33333333]
Для состояния [1 0 2] мин. интенсивности обслуживания = [0.3        0.26666667 0.66666667]
Для состояния [1 1 1] мин. интенсивности обслуживания = [0.3 0.4 0.5]
Для состояния [2 0 1] мин. интенсивности обслуживания = [0.4        0.26666667 0.5       ]
Для состояния [2 1 0] мин. интенсивности обслуживания = [0.4        0.4        0.33333333]
Для состояния [3 0 0] мин. интенсивности обслуживания = [0.5        0.26666667 0.33333333]
Для состояния [0 3 0] мин. интенсивности обслуживания = [0.2        0.66666667 0.33333333]
Для состояния [0 0 3] мин. интенсивности обслуживания = [0.2        0.26666667 0.83333333]
